
## Train Inverse Model (Synthetic Data, from output constraints optimization)

* Synthetic Data gen: gradient based input optimization from constrained outputs 
* invert inputs and outputs and train

## Web application 

Run interactively on the web at: 


## Inputs


* desired output requirements 

Raceway flame temp > 1900K.....(19, 'o_fta')
 
Hot metal temp > 1700K........(21, 'o_hmt')

Top gas temp > 100C,,,,,,,,,,,(23, 'o_tgt')

production rate ..............(39, 'o_prod_rt')


## Outputs



(2, 'i_h2i_rate')

(3, 'i_h2_temp')

(4, 'i_ngi_rate')

(6, 'i_pci_rate')

(11, 'i_o2_volfract')

(12, 'i_hbtemp')

(13, 'i_wind_rt')











## Installs


In [1]:

## !pip install xgboost
## !pip install onnxmltools
## !pip install onnxruntime
## !pip install pandas
## !pip install scikit-learn
## !pip install matplotlib
## !pip install seaborn
## !pip install mlxtend
## ! pip install skl2onnx



## Imports


In [2]:

import torch
import numpy as np
import pandas as pd
import sklearn
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import math
import seaborn as sns
import random
import functorch


import torch.optim as optim


from numpy.random import normal
from scipy.stats import norm
import scipy.stats as stats
from numpy import hstack
from numpy import vstack
from numpy import exp
from sklearn.neighbors import KernelDensity

##from typing import Optional

from torch import Tensor


from torch.utils.data import TensorDataset, DataLoader

from mlxtend.plotting import heatmap

## coefficient of determination (R**2)
from sklearn.metrics import r2_score

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [3]:

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster


from zipfile import ZipFile
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)


# Import the ONNX runtime environment
import onnxruntime as rt



In [4]:

import PDFshapingUtils as PDF_tk



## Instantiate PDF shaping and regression class


In [5]:

PDFshapingOBJ = PDF_tk.PDFshapingUtils()



## Initialize parameters


In [6]:

PDFshapingOBJ.N_error_range = 5         ## 20
PDFshapingOBJ.mean_impulse  = 0.0       ## -7.0      ## 0.0
PDFshapingOBJ.std_impulse   = 0.001     ## 1.0
PDFshapingOBJ.kde_std       = 0.5       ## 0.1       ## 0.5      ## 3.0  ## smaller is better approximation?
PDFshapingOBJ.bandwidth     = 0.2       ## 0.2->more jagged, 2.0->more smooth


In [7]:

PDFshapingOBJ.initializeImpulseGaussian()



## Load Data


In [8]:

df_synthetic_data_for_inverse = PDFshapingOBJ.load_synth_folder_to_pandas("SyntheticCorpus", flatten=True)


In [9]:

PDFshapingOBJ.CFD_raw_data = df_synthetic_data_for_inverse 


In [10]:

PDFshapingOBJ.headers_list = PDFshapingOBJ.CFD_raw_data.columns.values.tolist()


In [11]:

PDFshapingOBJ.CFD_raw_data


,clamp_min_0,clamp_min_1,clamp_min_2,clamp_min_3,clamp_min_4,clamp_min_5,clamp_min_6,final_input_0,final_input_1,final_input_2,...,clamp_max_6,lower_bounds_glob_0,lower_bounds_glob_1,lower_bounds_glob_2,final_output_0,final_output_1,final_output_2,upper_bounds_glob_0,upper_bounds_glob_1,upper_bounds_glob_2
0,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-15.408000,600.830994,110.005997,...,300.0,2358.701172,1655.914551,60.293190,2449.599121,1821.146973,79.871002,2491.664795,2166.322998,99.857727
1,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-4.730000,858.596985,56.514000,...,300.0,2317.072510,1092.126099,79.248924,2374.600098,1378.092041,95.074997,2500.922607,1712.875244,120.109390
2,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-22.931999,417.941986,25.250999,...,300.0,2188.174316,1385.928955,109.405258,2233.093994,1664.880981,122.009003,2296.180664,2026.555420,144.704681
3,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-18.221001,486.447998,29.957001,...,300.0,2078.979248,1005.422363,103.999542,2183.361084,1269.423950,106.377998,2326.123535,1619.734863,115.923889
4,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,21.247999,811.328979,117.718002,...,300.0,2519.401367,1051.182861,41.449707,2592.281982,1219.651978,47.130001,2705.451416,1472.596924,54.678768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,37.507000,665.169006,145.764008,...,300.0,2143.092773,136.790649,52.153313,2244.035889,270.920990,59.862999,2356.703125,431.092896,74.693016
252,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-15.446000,604.286011,76.025002,...,300.0,2128.074463,1482.573486,147.429230,2147.043945,1720.956055,149.908005,2190.985840,1997.258301,176.805908
253,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-16.747000,755.739014,90.889999,...,300.0,2471.232422,1957.027832,98.135017,2494.469971,2247.897949,102.805000,2557.979980,2453.308838,122.096336
254,-50.0,-50.0,-50.0,-50.0,-29.0,1150.0,100.0,-10.564000,656.293030,172.195999,...,300.0,2104.450684,1413.251709,113.882812,2150.665039,1582.086060,123.663002,2225.944824,1712.263672,147.003464


In [12]:

## PDFshapingOBJ.read_csv_file_with_pandas('CFD.14.2025_REVISED.csv')


In [13]:

## PDFshapingOBJ.CFD_raw_data


In [14]:

PDFshapingOBJ.print_headers_list()


['clamp_min_0', 'clamp_min_1', 'clamp_min_2', 'clamp_min_3', 'clamp_min_4', 'clamp_min_5', 'clamp_min_6', 'final_input_0', 'final_input_1', 'final_input_2', 'final_input_3', 'final_input_4', 'final_input_5', 'final_input_6', 'clamp_max_0', 'clamp_max_1', 'clamp_max_2', 'clamp_max_3', 'clamp_max_4', 'clamp_max_5', 'clamp_max_6', 'lower_bounds_glob_0', 'lower_bounds_glob_1', 'lower_bounds_glob_2', 'final_output_0', 'final_output_1', 'final_output_2', 'upper_bounds_glob_0', 'upper_bounds_glob_1', 'upper_bounds_glob_2']
30
(0, 'clamp_min_0')
(1, 'clamp_min_1')
(2, 'clamp_min_2')
(3, 'clamp_min_3')
(4, 'clamp_min_4')
(5, 'clamp_min_5')
(6, 'clamp_min_6')
(7, 'final_input_0')
(8, 'final_input_1')
(9, 'final_input_2')
(10, 'final_input_3')
(11, 'final_input_4')
(12, 'final_input_5')
(13, 'final_input_6')
(14, 'clamp_max_0')
(15, 'clamp_max_1')
(16, 'clamp_max_2')
(17, 'clamp_max_3')
(18, 'clamp_max_4')
(19, 'clamp_max_5')
(20, 'clamp_max_6')
(21, 'lower_bounds_glob_0')
(22, 'lower_bounds_glob

In [15]:
'''


PDFshapingOBJ.list_of_selected_column_names = ['i_h2i_rate','i_h2_temp','i_ngi_rate','i_ng_temp','i_pci_rate',
                    'i_wpi_rate','i_o2_volfract',
                    'i_hbtemp','i_wind_rt','o_prod_rt', 'o_tgt', 'o_hmt','o_fta']



PDFshapingOBJ.print_correlation_coefficients()

'''


"\n\n\nPDFshapingOBJ.list_of_selected_column_names = ['i_h2i_rate','i_h2_temp','i_ngi_rate','i_ng_temp','i_pci_rate',\n                    'i_wpi_rate','i_o2_volfract',\n                    'i_hbtemp','i_wind_rt','o_prod_rt', 'o_tgt', 'o_hmt','o_fta']\n\n\n\nPDFshapingOBJ.print_correlation_coefficients()\n\n"

In [16]:

PDFshapingOBJ.convert_pd_data_to_numpy()


[[ -50.          -50.          -50.         ... 2491.66479492
  2166.32299805   99.85772705]
 [ -50.          -50.          -50.         ... 2500.92260742
  1712.87524414  120.10939026]
 [ -50.          -50.          -50.         ... 2296.18066406
  2026.55541992  144.7046814 ]
 ...
 [ -50.          -50.          -50.         ... 2557.97998047
  2453.30883789  122.09633636]
 [ -50.          -50.          -50.         ... 2225.94482422
  1712.26367188  147.00346375]
 [ -50.          -50.          -50.         ... 2599.23950195
  2391.33496094  134.36517334]]
(256, 30)



## Select inputs and outputs


In [17]:

PDFshapingOBJ.gen_X_y_for_selected_indeces(  
                   inputs = [ 21,22,23,27,28,29 ] ,  
                   outputs= [  7, 8, 9, 10, 11, 12, 13 ]   
)


6
7


In [18]:

print(PDFshapingOBJ.X.shape)
print(PDFshapingOBJ.y.shape)


(256, 6)
(256, 7)


In [19]:

PDFshapingOBJ.random_seed = int( random.random() * 100  )         ## defautl is 42


In [20]:

PDFshapingOBJ.split_np_data_train_test(selected_test_size=0.2)


(204, 6)
(52, 6)
(204, 7)
(52, 7)


In [21]:

PDFshapingOBJ.convert_dataset_from_np_to_torch()


In [22]:

PDFshapingOBJ.standardize_X_scales()
PDFshapingOBJ.standardize_y_scales()


In [23]:

PDFshapingOBJ.gen_Dataloader_train()

PDFshapingOBJ.train_dl


In [24]:

n_inputs  = 6
n_outputs = 7

class ResidualNet(nn.Module):
    def __init__(self, x_means, x_deviations, y_means, y_deviations, dropout_rate=0.1, device='cuda'):
        
        super().__init__()
        self.device = device
        
        self.x_means      = x_means.to(self.device)
        self.x_deviations = x_deviations.to(self.device)
        self.y_means      = y_means.to(self.device)
        self.y_deviations = y_deviations.to(self.device)

     
        self.fc1  = nn.Linear(n_inputs, 64)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(64, 64)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(64, n_outputs)

        # If input_dim ≠ 64, project for residual addition
        self.input_proj = nn.Linear(n_inputs, 64) if n_inputs != 64 else nn.Identity()
        
        # Move model to device
        self.to(self.device)
        

    def forward(self, x):
        x  = x.to(self.device)
        x  = (x - self.x_means) / self.x_deviations
        x0 = self.input_proj(x)
        x1 = self.dropout1(self.act1(self.fc1(x)))
        x2 = self.dropout2(self.act2(self.fc2(x1)))
        x_res = x2 + x0
        y_scaled = self.fc3(x_res)
        y_descaled = y_scaled * self.y_deviations + self.y_means
        
        return y_descaled, y_scaled
    
    


In [25]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model     = ResidualNet(
             PDFshapingOBJ.x_means, 
             PDFshapingOBJ.x_deviations, 
             PDFshapingOBJ.y_means, 
             PDFshapingOBJ.y_deviations,
             device=device  # pass device into the model
    
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.MSELoss()




In [26]:

model.train()

for epoch in range(1000):
    for xb, yb in PDFshapingOBJ.train_dl:
        
        xb = xb.to(model.device)
        yb = yb.to(model.device)
        
        optimizer.zero_grad()
        pred_descaled, pred_scaled = model(xb)
        loss = loss_fn(pred_scaled, yb)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(epoch, ".....", loss.item() ) 
    


    

0 ..... 1.3519436120986938
100 ..... 0.33797410130500793
200 ..... 0.5971142053604126
300 ..... 0.2757813036441803
400 ..... 0.19908137619495392
500 ..... 0.21069712936878204
600 ..... 0.21734409034252167
700 ..... 0.15136085450649261
800 ..... 0.23077301681041718
900 ..... 0.17864631116390228


In [27]:

print("=== SANITY CHECK: SCALING ===")
print("y_means:", PDFshapingOBJ.y_means)
print("y_deviations:", PDFshapingOBJ.y_deviations)

print("===============================================")

print("x_means:", PDFshapingOBJ.x_means)
print("x_deviations:", PDFshapingOBJ.x_deviations)



=== SANITY CHECK: SCALING ===
y_means: tensor([[   1.7310,  627.3831,  100.1391,   96.1464,   30.3192, 1345.2699,
          217.2982]])
y_deviations: tensor([[ 18.1763, 157.9689,  48.8911,  43.1962,   7.7842,  24.0978,  33.1882]])
x_means: tensor([[2282.4824, 1227.7366,   68.6166, 2423.3914, 1621.1455,   95.0748]])
x_deviations: tensor([[207.2598, 512.7145,  40.7899, 212.3609, 543.6526,  43.1857]])



## Test model


In [28]:

print( PDFshapingOBJ.X_test_tr[0] )
print( PDFshapingOBJ.y_test_tr[0] )


tensor([2184.4797, 2202.6628,  132.9694, 2414.4944, 2885.2183,  167.2353])
tensor([ -16.3880,  674.5480,  127.5450,    4.8760,   27.3430, 1347.1010,
         213.6570])


In [29]:


print( PDFshapingOBJ.X_test_tr.shape )
print( PDFshapingOBJ.y_test_tr.shape )

with torch.no_grad():
    
    pred_test_descaled, pred_test_scaled = model(  PDFshapingOBJ.X_test_tr  )


torch.Size([52, 6])
torch.Size([52, 7])


In [30]:


y_true = PDFshapingOBJ.y_test_tr.detach().cpu().numpy()

y_pred = pred_test_descaled.detach().cpu().numpy()  # or use descaled pair

mae  = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
rmse = mean_squared_error( y_true, y_pred, squared=False, multioutput='raw_values')
r2   = r2_score(           y_true, y_pred, multioutput='raw_values')

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)
print("Overall averages:", 
      mae.mean(), rmse.mean(), r2.mean())



MAE: [  9.631208  101.93179    48.117775   41.700523    3.1454396  19.641449
  17.965858 ]
RMSE: [ 11.78431  128.04898   67.17397   54.788925   4.306202  28.003965
  25.208857]
R2: [ 0.61687375  0.55699547 -0.93957544 -0.20274108  0.6581342   0.05378012
  0.53081157]
Overall averages: 34.59058 45.616463 0.18203979837320486


In [31]:

r2   = r2_score(           y_true, y_pred, multioutput='raw_values')
r2


array([ 0.61687375,  0.55699547, -0.93957544, -0.20274108,  0.6581342 ,
        0.05378012,  0.53081157])

In [32]:

r2.mean()


0.18203979837320486

In [33]:

print('Testing R**2 - descaled:', r2_score( y_pred, y_true   )  )


Testing R**2 - descaled: 0.08096772054611692


In [34]:

print('Testing R**2 - descaled:', r2_score( y_true,  y_pred   )  )


Testing R**2 - descaled: 0.18203979837320486


In [35]:

PDFshapingOBJ.r2_score_torch(PDFshapingOBJ.y_test_tr.to(PDFshapingOBJ.device), pred_test_descaled).mean()


tensor(0.1820, device='cuda:0')


## ONNX


In [36]:

import torch

# Your trained model
model.eval().float()


dummy_input = torch.randn(1, 6, dtype=torch.float32)

# Export to ONNX (single output called "output1")
torch.onnx.export(
    model,
    dummy_input,
    "ONNXmodels/resNet_Inverse_syntheticData.onnx",
    input_names=["input1"],
    output_names=["output1"],
    opset_version=15,              # fine for onnxruntime-web
    do_constant_folding=True,
    dynamic_axes={
        "input1": {0: "batch"},
        "output1": {0: "batch"}
    }
)
print("ONNX model saved")



ONNX model saved



## XGBoost


In [37]:

regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)


In [38]:

regressor.fit(PDFshapingOBJ.X_train, PDFshapingOBJ.y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [39]:

print( len(PDFshapingOBJ.X_train  ))
print( len(PDFshapingOBJ.y_train  ))


204
204


In [40]:

print( PDFshapingOBJ.X_train.shape  )
print( PDFshapingOBJ.y_train.shape  )


(204, 6)
(204, 7)


In [41]:

y_pred = regressor.predict(PDFshapingOBJ.X_test )


In [42]:

y_pred


array([[ -18.775524  ,  568.96704   ,   79.11125   ,   93.173744  ,
          29.89427   , 1328.2002    ,  248.4155    ],
       [ -15.100213  ,  725.49133   ,   80.513794  ,  133.29036   ,
          20.701176  , 1351.8484    ,  252.7776    ],
       [   0.24485254,  744.1457    ,   89.809296  ,  131.1893    ,
          26.434008  , 1353.8689    ,  247.17802   ],
       [  13.808264  ,  691.5573    ,   89.679565  ,  129.16666   ,
          40.429897  , 1351.4054    ,  190.56238   ],
       [  -1.0432308 ,  547.9631    ,  110.154884  ,   77.40523   ,
          29.570608  , 1315.199     ,  229.47792   ],
       [   5.0077014 ,  536.85706   ,   90.1988    ,   89.347435  ,
          36.60099   , 1334.2969    ,  187.07762   ],
       [ -20.088669  ,  543.4079    ,  141.20224   ,   43.006004  ,
          23.199774  , 1362.0105    ,  241.41803   ],
       [ -22.010288  ,  757.65186   ,   94.14734   ,   78.45734   ,
          27.232756  , 1356.3805    ,  224.14838   ],
       [ -10.543886  ,  

In [43]:

r2_score( PDFshapingOBJ.y_test, y_pred )


0.45329396905971686